In [ ]:
# 1. 基础重建（保留所有embedding缓存）
from mcp_embedding_manager import MCPEmbeddingManager
from collections import OrderedDict

manager = MCPEmbeddingManager()
manager.build_index(
    tool_registry_path="mcp_generated_library/tool_registry_consolidated.json",
    force_rebuild=True
)

# 2. 查看缓存统计
stats = manager.get_cache_stats()
print(f"📊 Cache Statistics:")
print(f"  - Embedding cache: {stats['embedding_cache']['entries']} entries ({stats['embedding_cache']['size_mb']:.2f} MB)")
print(f"  - Search cache: {stats['search_cache']['entries']} entries")
print(f"  - Total size: {stats['total_size_mb']:.2f} MB")

# 3. 清理搜索缓存但保留embedding
manager.clear_cache(keep_embeddings=True)

# 4. 完全清理（仅在必要时）
# manager.clear_cache(keep_embeddings=False)

2025-07-11 17:40:16,822 - mcp_embedding_manager - INFO - Loaded search cache with 3 entries


[MCPEmbeddingManager] Initializing with unified API client manager
[MCPEmbeddingManager] Using embedding model: text-embedding-3-large
[MCPEmbeddingManager] Client initialized successfully
[OperationEmbeddingIndex] Initializing with unified API client manager
[OperationEmbeddingIndex] OpenAI client initialized with model: gpt-4o-mini
[OperationEmbeddingIndex] Using embedding model: text-embedding-3-large
[OperationEmbeddingIndex] Detecting actual embedding dimension...


2025-07-11 17:40:17,110 - httpx - INFO - HTTP Request: POST https://archer222arc.openai.azure.com/openai/deployments/text-embedding-3-large/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2025-07-11 17:40:17,201 - mcp_embedding_manager - INFO - Initialized operation mappings with 142 terms
2025-07-11 17:40:17,278 - mcp_embedding_manager - INFO - Building tool embedding index...
2025-07-11 17:40:17,280 - mcp_embedding_manager - INFO - Creating embeddings for 30 tools...


[OperationEmbeddingIndex] Detected embedding dimension: 3072
[INFO] Loaded 6050 embeddings from persistent cache
[OperationEmbeddingIndex] Initialized with 6050 cached embeddings
[INFO] Loaded 14 LLM-enhanced operation definitions from cache
[INFO] Found cached operation index at .mcp_operation_cache/operation_index.pkl
[INFO] Loading operation index from .mcp_operation_cache/operation_index.pkl
[INFO] Successfully loaded FAISS index with dimension 3072
[INFO] Operation index loaded successfully from .mcp_operation_cache/operation_index.pkl
[INFO] Loaded 14 operations with dimension 3072
[INFO] Successfully loaded cached index
[Cache] Loaded 4010 entries from file


  0%|          | 0/30 [00:00<?, ?it/s]2025-07-11 17:40:17,393 - httpx - INFO - HTTP Request: POST https://archer222arc.openai.azure.com/openai/deployments/text-embedding-3-large/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2025-07-11 17:40:17,480 - httpx - INFO - HTTP Request: POST https://archer222arc.openai.azure.com/openai/deployments/text-embedding-3-large/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2025-07-11 17:40:17,615 - httpx - INFO - HTTP Request: POST https://archer222arc.openai.azure.com/openai/deployments/text-embedding-3-large/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2025-07-11 17:40:17,704 - httpx - INFO - HTTP Request: POST https://archer222arc.openai.azure.com/openai/deployments/text-embedding-3-large/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
  3%|▎         | 1/30 [00:00<00:12,  2.38it/s]2025-07-11 17:40:17,784 - httpx - INFO - HTTP Request: POST https://archer222arc.openai.azure.com/openai/deployments

[MCPEmbeddingManager] Expected dimension: 3072
[MCPEmbeddingManager] Actual embedding dimension: 3072
[INFO] Saved 4010 search cache entries
📊 Cache Statistics:
  - Embedding cache: 100 entries (2.35 MB)
  - Search cache: 4010 entries
  - Total size: 6.02 MB
🗑️ Clearing cache (keep_embeddings=True)...
  - Cleared search cache
  - Kept 100 embeddings


: 

In [9]:
import torch
import json
import os
from pathlib import Path
import numpy as np

# 设置checkpoint路径
checkpoint_dir = Path("checkpoints")  # 根据你的实际路径修改
print(f"Checkpoint目录: {checkpoint_dir}")
print(f"目录是否存在: {checkpoint_dir.exists()}")

# 列出所有checkpoint文件
if checkpoint_dir.exists():
    checkpoints = list(checkpoint_dir.glob("*.pt"))
    print(f"\n找到 {len(checkpoints)} 个checkpoint文件:")
    for ckpt in sorted(checkpoints):
        print(f"  - {ckpt.name}")
else:
    print("❌ Checkpoint目录不存在!")
    checkpoints = []

# 分析最新的checkpoint
if checkpoints:
    # 选择要分析的checkpoint
    # 可以选择 best_model.pt 或最新的 checkpoint_episode_xxx.pt
    best_model = checkpoint_dir / "best_model.pt"
    # best_model = checkpoint_dir / "checkpoint_episode_400.pt"
    if best_model.exists():
        target_checkpoint = best_model
    else:
        # 获取最新的checkpoint
        target_checkpoint = max(checkpoints, key=lambda x: x.stat().st_mtime)
    
    print(f"\n分析checkpoint: {target_checkpoint.name}")
    print("=" * 60)
    
    # 加载checkpoint
    checkpoint = torch.load(target_checkpoint, map_location='cpu')
    
    # 1. 打印基本信息
    print("\n📊 基本信息:")
    if 'metadata' in checkpoint:
        metadata = checkpoint['metadata']
        print(f"  Episode: {metadata.get('episode', 'N/A')}")
        print(f"  Success Rate: {metadata.get('success_rate', 0):.2%}")
        print(f"  Timestamp: {metadata.get('timestamp', 'N/A')}")
        print(f"  Best Success Rate: {metadata.get('best_success_rate', 0):.2%}")
    else:
        print("  ⚠️ 没有metadata信息")
    
    # 2. 训练历史
    print("\n📈 训练历史:")
    if 'training_history' in checkpoint:
        history = checkpoint['training_history']
        if 'success' in history and history['success']:
            success_rates = history['success']
            print(f"  总Episodes: {len(success_rates)}")
            print(f"  最后10个episode成功率: {np.mean(success_rates[-10:]):.2%}")
            print(f"  最高成功率: {max(success_rates):.2%}")
            print(f"  成功的episodes数: {sum(success_rates)}")
            
            # 打印最后几个episode的成功情况
            print(f"\n  最后20个episode成功情况:")
            last_20 = success_rates[-20:] if len(success_rates) >= 20 else success_rates
            success_str = "".join(["✓" if s else "✗" for s in last_20])
            print(f"  {success_str}")
        else:
            print("  ⚠️ 没有成功率记录")
            
        if 'rewards' in history and history['rewards']:
            rewards = history['rewards']
            print(f"\n  平均奖励: {np.mean(rewards):.2f}")
            print(f"  最后10个episode平均奖励: {np.mean(rewards[-10:]):.2f}")
            print(f"  最高奖励: {max(rewards):.2f}")
    else:
        print("  ⚠️ 没有训练历史")
    
    # 3. 配置信息
    print("\n⚙️ 训练配置:")
    if 'config' in checkpoint:
        config = checkpoint['config']
        print(f"  算法: {config.get('algorithm', 'N/A')}")
        print(f"  学习率: {config.get('learning_rate', 'N/A')}")
        print(f"  批大小: {config.get('batch_size', 'N/A')}")
        print(f"  最大episode长度: {config.get('max_episode_length', 'N/A')}")
        print(f"  课程学习: {config.get('use_curriculum', False)}")
    
    # 4. 额外信息
    print("\n📦 其他信息:")
    all_keys = list(checkpoint.keys())
    print(f"  Checkpoint包含的所有键: {all_keys}")
    
    # 检查任务统计
    if 'task_statistics' in checkpoint:
        print("\n  任务统计:")
        task_stats = checkpoint['task_statistics']
        for task_type, stats in task_stats.items():
            print(f"    {task_type}: 成功率 {stats.get('success_rate', 0):.2%}")
    
    # 检查课程阶段
    if 'curriculum_stage' in checkpoint:
        print(f"\n  课程阶段: Stage {checkpoint['curriculum_stage']}")
    
    # 检查是否有工具关键性数据
    criticality_file = checkpoint_dir / "tool_criticality.json"
    if criticality_file.exists():
        print("\n  ✓ 找到tool_criticality.json文件")
        with open(criticality_file, 'r') as f:
            criticality_data = json.load(f)
            print(f"    记录的工具数: {len(criticality_data.get('tool_criticality', {}))}")
    
    # 5. 诊断信息
    print("\n🔍 可能的问题诊断:")
    if 'metadata' in checkpoint:
        success_rate = checkpoint['metadata'].get('success_rate', 0)
        if success_rate == 0:
            print("  ❌ 成功率为0，可能的原因：")
            print("    1. 任务太难或奖励设计有问题")
            print("    2. 探索不足，陷入局部最优")
            print("    3. required_tools定义可能有问题")
            print("    4. 最大步数限制太严格")
            print("    5. 课程学习进度太快")
            
            # 检查episode长度
            if 'training_history' in checkpoint and 'lengths' in checkpoint['training_history']:
                avg_length = np.mean(checkpoint['training_history']['lengths'])
                max_allowed = config.get('max_episode_length', 100)
                if avg_length > max_allowed * 0.9:
                    print(f"\n    ⚠️ 平均episode长度({avg_length:.1f})接近最大限制({max_allowed})!")
                    print("       可能需要增加max_episode_length")
else:
    print("\n❌ 没有找到任何checkpoint文件!")
    print("请检查:")
    print("1. checkpoint路径是否正确")
    print("2. 训练是否正常保存了checkpoint")

Checkpoint目录: checkpoints
目录是否存在: True

找到 6 个checkpoint文件:
  - best_model.pt
  - checkpoint_episode_0.pt
  - checkpoint_episode_100.pt
  - final_cpu_optimized.pt
  - final_gpu_model.pt
  - final_model.pt

分析checkpoint: best_model.pt


/tmp/ipykernel_199909/167464570.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(target_checkpoint, map_location='cpu')



📊 基本信息:
  ⚠️ 没有metadata信息

📈 训练历史:
  总Episodes: 200
  最后10个episode成功率: 0.00%
  最高成功率: 0.00%
  成功的episodes数: 0.0

  最后20个episode成功情况:
  ✗✗✗✗✗✗✗✗✗✗✗✗✗✗✗✗✗✗✗✗

  平均奖励: -286.05
  最后10个episode平均奖励: -315.50
  最高奖励: -60.00

⚙️ 训练配置:
  算法: ppo
  学习率: 0.0005
  批大小: 1024
  最大episode长度: 100
  课程学习: True

📦 其他信息:
  Checkpoint包含的所有键: ['algorithm', 'state_dim', 'action_dim', 'episode', 'config', 'timestamp', 'use_task_aware_state', 'enforce_workflow', 'use_phase2_scoring', 'cpu_optimized', 'network_state_dict', 'model_state_dict', 'optimizer_state_dict', 'training_steps', 'training_history', 'best_success_rate']

🔍 可能的问题诊断:


In [ ]:
# pip install azure-ai-inference
import os
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential

# api_key = os.getenv("AZURE_INFERENCE_CREDENTIAL", '')
api_key = "9wiSC2YySp6iDFL45NIPuoJ9Ynm2CcEjPw4FDjGAeCOpRdZjdetdJQQJ99BGACYeBjFXJ3w3AAABACOGqpWV"
if not api_key:
  raise Exception("A key should be provided to invoke the endpoint")

client = ChatCompletionsClient(
    endpoint='https://archer222arc.openai.azure.com/openai/deployments/gpt-4o-mini',
    credential=AzureKeyCredential(api_key),
    
)

payload = {
  "messages": [
    {
      "role": "user",
      "content": "I am going to Paris, what should I see?"
    },
    {
      "role": "assistant",
      "content": "Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are some of the top attractions to see in Paris:\n\n1. The Eiffel Tower: The iconic Eiffel Tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city.\n2. The Louvre Museum: The Louvre is one of the world's largest and most famous museums, housing an impressive collection of art and artifacts, including the Mona Lisa.\n3. Notre-Dame Cathedral: This beautiful cathedral is one of the most famous landmarks in Paris and is known for its Gothic architecture and stunning stained glass windows.\n\nThese are just a few of the many attractions that Paris has to offer. With so much to see and do, it's no wonder that Paris is one of the most popular tourist destinations in the world."
    },
    {
      "role": "user",
      "content": "What is so great about #1?"
    }
  ],
  "max_tokens": 4096,
  "temperature": 1,
  "top_p": 1,
  "stop": []
}
response = client.complete(payload)

print("Response:", response.choices[0].message.content)
print("Model:", response.model)
print("Usage:")
print("	Prompt tokens:", response.usage.prompt_tokens)
print("	Total tokens:", response.usage.total_tokens)
print("	Completion tokens:", response.usage.completion_tokens)

Exception: A key should be provided to invoke the endpoint

In [ ]:
import os
from openai import AzureOpenAI
endpoint = "https://archer222arc.openai.azure.com/"
model_name = "gpt-4o-mini"
deployment = "gpt-4o-mini"

client = AzureOpenAI(
    api_version="2024-12-01-preview",
    azure_endpoint="https://archer222arc.openai.azure.com/",
    api_key="9wiSC2YySp6iDFL45NIPuoJ9Ynm2CcEjPw4FDjGAeCOpRdZjdetdJQQJ99BGACYeBjFXJ3w3AAABACOGqpWV",
)
response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "I am going to Paris, what should I see?",
        }
    ],
    max_tokens=4096,
    temperature=1.0,
    top_p=1.0,
    model=deployment
)

print(response.choices[0].message.content)

2025-07-11 09:23:21,469 - httpx - INFO - HTTP Request: POST https://archer222arc.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


Paris is a vibrant city with a rich history, stunning architecture, and a diverse array of attractions. Here’s a list of must-see places and experiences you shouldn’t miss:

### Iconic Landmarks:
1. **Eiffel Tower** - You can't visit Paris without seeing this iconic structure. Consider going at sunset for breathtaking views.
2. **Louvre Museum** - Home to thousands of art pieces, including the Mona Lisa and Venus de Milo; plan your visit as it can be large.
3. **Notre-Dame Cathedral** - Though currently under restoration, the exterior and the Île de la Cité are worth seeing.
4. **Sacré-Cœur Basilica** - Located on Montmartre Hill, it offers stunning views of the city and a beautiful interior.
5. **Arc de Triomphe** - Climb to the top for another great view of the city, and explore the Champs-Élysées.

### Museums and Cultural Venues:
6. **Musée d’Orsay** - Housed in a former railway station, it features Impressionist masterpieces.
7. **Centre Pompidou** - Known for its modern art colle

In [10]:
# %% [markdown]
# # 创建 Operations Embedding 缓存
# 这个 notebook 用于生成新的 operations embedding 缓存文件，包括 LLM 增强的操作定义

# %% 导入必要的库
import os
import json
import shutil
from pathlib import Path
from datetime import datetime

# 确保在正确的目录下运行
print(f"当前工作目录: {os.getcwd()}")

# %% 清理旧缓存（可选）
def clean_old_cache(backup=True):
    """清理旧的缓存文件"""
    cache_dir = Path(".mcp_operation_cache")
    
    if not cache_dir.exists():
        print("缓存目录不存在，无需清理")
        return
    
    if backup:
        # 创建备份
        backup_dir = Path(f".mcp_operation_cache_backup_{datetime.now().strftime('%Y%m%d_%H%M%S')}")
        shutil.copytree(cache_dir, backup_dir)
        print(f"已备份旧缓存到: {backup_dir}")
    
    # 删除特定文件
    files_to_remove = [
        "operation_index.pkl",
        "llm_operation_definitions.json",
        "embedding_cache.pkl"
    ]
    
    for file_name in files_to_remove:
        file_path = cache_dir / file_name
        if file_path.exists():
            file_path.unlink()
            print(f"已删除: {file_path}")
    
    print("缓存清理完成")

# 运行清理（根据需要取消注释）
# clean_old_cache(backup=True)

# %% 初始化 Operation Embedding Index
from operation_embedding_index import OperationEmbeddingIndex, get_operation_index

print("初始化 Operation Embedding Index...")
# 创建新实例，强制重建索引
index = OperationEmbeddingIndex(use_cache=True)

# %% 查看生成的操作定义
print("\n=== 操作定义统计 ===")
print(f"总操作数: {len(index.operation_definitions)}")
print("\n按类别分组:")

# 统计各类别的操作数
category_stats = {}
for op_name, op_def in index.operation_definitions.items():
    category = op_def.get('category', 'unknown')
    if category not in category_stats:
        category_stats[category] = []
    category_stats[category].append(op_name)

for category, operations in sorted(category_stats.items()):
    print(f"\n{category.upper()} ({len(operations)} operations):")
    for op in sorted(operations):
        print(f"  - {op}")

# %% 查看详细的操作定义
print("\n=== 详细操作定义示例 ===")

# 显示每个类别的一个示例
for category, operations in category_stats.items():
    if operations:
        op_name = operations[0]
        op_def = index.operation_definitions[op_name]
        print(f"\n{category.upper()} 示例 - {op_name}:")
        print(f"  描述: {op_def['description']}")
        print(f"  同义词: {', '.join(op_def['synonyms'][:5])}...")
        if 'related_operations' in op_def:
            print(f"  相关操作: {', '.join(op_def.get('related_operations', [])[:3])}...")

# %% 测试语义搜索功能
print("\n=== 测试语义搜索 ===")

test_queries = [
    "read data from csv file",
    "validate json schema",
    "merge multiple datasets",
    "connect to REST API",
    "calculate statistical metrics",
    "save results to database",
    "monitor system performance",
    "parse XML documents"
]

for query in test_queries:
    print(f"\n查询: '{query}'")
    results = index.search_operation(query, k=3)
    for op_name, score in results:
        op_def = index.operation_embeddings.get(op_name)
        if op_def:
            print(f"  - {op_name} ({op_def.category}): {score:.3f}")

# %% 保存缓存状态
print("\n=== 缓存文件状态 ===")

cache_dir = Path(".mcp_operation_cache")
if cache_dir.exists():
    for file_path in cache_dir.iterdir():
        if file_path.is_file():
            size_kb = file_path.stat().st_size / 1024
            print(f"{file_path.name}: {size_kb:.2f} KB")

# %% 验证 LLM 增强的定义
llm_def_path = Path(".mcp_operation_cache/llm_operation_definitions.json")
if llm_def_path.exists():
    with open(llm_def_path, 'r') as f:
        llm_definitions = json.load(f)
    
    print(f"\n=== LLM 增强定义验证 ===")
    print(f"总操作数: {len(llm_definitions)}")
    
    # 查找新增的操作（不在基础定义中的）
    base_operations = {'read', 'validate', 'transform', 'aggregate', 'write', 'calculate', 'connect', 'log'}
    new_operations = [op for op in llm_definitions.keys() if op not in base_operations]
    
    if new_operations:
        print(f"\nLLM 生成的新操作 ({len(new_operations)}):")
        for op in new_operations[:10]:  # 只显示前10个
            print(f"  - {op}: {llm_definitions[op]['description'][:60]}...")

# %% 导出统计信息
stats = index.get_stats()
print(f"\n=== 索引统计信息 ===")
print(json.dumps(stats, indent=2))

# %% 手动触发缓存保存
print("\n保存所有缓存...")
index._save_embedding_cache()
cache_path = index._get_cache_path()
index.save_index(cache_path)
print("缓存保存完成！")

# %% 创建操作定义的可读报告
report_path = Path("operation_definitions_report.json")
report = {
    "generated_at": datetime.now().isoformat(),
    "total_operations": len(index.operation_definitions),
    "categories": category_stats,
    "definitions": index.operation_definitions,
    "statistics": stats
}

with open(report_path, 'w') as f:
    json.dump(report, f, indent=2)
print(f"\n操作定义报告已保存到: {report_path}")

# %% [markdown]
# ## 下一步
# 
# 1. 检查生成的缓存文件是否正确
# 2. 运行其他使用 operation index 的模块，确认兼容性
# 3. 如果需要重新生成，可以删除缓存文件并重新运行此 notebook
# 
# ### 缓存文件位置
# - `.mcp_operation_cache/operation_index.pkl` - 主索引文件
# - `.mcp_operation_cache/llm_operation_definitions.json` - LLM 增强的定义
# - `.mcp_operation_cache/embedding_cache.pkl` - 嵌入向量缓存

当前工作目录: /root/WorkflowBench/scale_up
初始化 Operation Embedding Index...
[OperationEmbeddingIndex] Initializing with unified API client manager
[OperationEmbeddingIndex] OpenAI client initialized with model: gpt-4o-mini
[OperationEmbeddingIndex] Using embedding model: text-embedding-3-small
[OperationEmbeddingIndex] Using embedding dimension: 1536
[INFO] Loaded 2973 embeddings from persistent cache
[OperationEmbeddingIndex] Initialized with 2973 cached embeddings
[INFO] Loaded 14 LLM-enhanced operation definitions from cache
[INFO] Loading operation index from cache
[INFO] Operation index loaded from .mcp_operation_cache/operation_index.pkl

=== 操作定义统计 ===
总操作数: 14

按类别分组:

AGGREGATION (1 operations):
  - aggregate

AUTOMATION (1 operations):
  - schedule

COMMUNICATION (1 operations):
  - notify

COMPUTATION (1 operations):
  - calculate

DATA TRANSFER (1 operations):
  - stream

INPUT (1 operations):
  - read

INTEGRATION (2 operations):
  - connect
  - expose

OUTPUT (1 operations):
  

2025-07-11 10:13:39,538 - httpx - INFO - HTTP Request: POST https://archer222arc.openai.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2025-07-11 10:13:39,670 - httpx - INFO - HTTP Request: POST https://archer222arc.openai.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


  - read (input): 0.347
  - parse (transformation): 0.300
  - write (output): 0.182

查询: 'validate json schema'
  - validate (validation): 0.360
  - parse (transformation): 0.232
  - transform (transformation): 0.226

查询: 'merge multiple datasets'


2025-07-11 10:13:39,834 - httpx - INFO - HTTP Request: POST https://archer222arc.openai.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2025-07-11 10:13:39,967 - httpx - INFO - HTTP Request: POST https://archer222arc.openai.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


  - aggregate (aggregation): 0.414
  - integrate (integration): 0.353
  - transform (transformation): 0.233

查询: 'connect to REST API'
  - integrate (integration): 0.244
  - validate (validation): 0.229
  - transform (transformation): 0.190

查询: 'calculate statistical metrics'


2025-07-11 10:13:40,101 - httpx - INFO - HTTP Request: POST https://archer222arc.openai.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2025-07-11 10:13:40,230 - httpx - INFO - HTTP Request: POST https://archer222arc.openai.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


  - compute (computation): 0.361
  - aggregate (aggregation): 0.257
  - validate (validation): 0.199

查询: 'save results to database'
  - write (output): 0.370
  - cache (utility): 0.301
  - transform (transformation): 0.221

查询: 'monitor system performance'


2025-07-11 10:13:40,356 - httpx - INFO - HTTP Request: POST https://archer222arc.openai.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2025-07-11 10:13:40,485 - httpx - INFO - HTTP Request: POST https://archer222arc.openai.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


  - integrate (integration): 0.260
  - compute (computation): 0.254
  - validate (validation): 0.207

查询: 'parse XML documents'
  - parse (transformation): 0.333
  - transform (transformation): 0.251
  - read (input): 0.225

=== 缓存文件状态 ===
llm_operation_definitions.json: 4.77 KB
operation_index.pkl: 196.93 KB
embedding_cache.pkl: 35890.82 KB

=== LLM 增强定义验证 ===
总操作数: 14

LLM 生成的新操作 (6):
  - filter: Filter data based on specific criteria or conditions...
  - schedule: Schedule operations to run at specific times or intervals...
  - notify: Send notifications or alerts based on events or conditions...
  - backup: Create a copy of data for recovery or preservation purposes...
  - stream: Continuously transfer data in real-time from one source to a...
  - expose: Expose data or services through an API for external access...

=== 索引统计信息 ===
{
  "total_operations": 10,
  "categories": {
    "input": 1,
    "transformation": 3,
    "validation": 1,
    "aggregation": 1,
    "computation": 1,


In [ ]:
#!/usr/bin/env python3
"""
检查OpenAI API余额和使用情况
"""

import os
import requests
from datetime import datetime, timedelta
from openai import OpenAI

def check_openai_usage():
    """检查OpenAI API的使用情况"""
    
    # 获取API密钥
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        print("错误：未找到OPENAI_API_KEY环境变量")
        return
    
    # 设置请求头
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    
    # 获取当前日期
    today = datetime.now()
    start_date = (today - timedelta(days=30)).strftime("%Y-%m-%d")
    end_date = today.strftime("%Y-%m-%d")
    
    print(f"正在检查 {start_date} 到 {end_date} 的使用情况...\n")
    
    # 检查订阅信息
    try:
        # 注意：OpenAI可能会更改这些端点
        subscription_url = "https://api.openai.com/v1/dashboard/billing/subscription"
        response = requests.get(subscription_url, headers=headers)
        
        if response.status_code == 200:
            data = response.json()
            print("=== 订阅信息 ===")
            print(f"计划类型: {data.get('plan', {}).get('title', 'Unknown')}")
            print(f"硬限制: ${data.get('hard_limit_usd', 0):.2f}")
            print(f"软限制: ${data.get('soft_limit_usd', 0):.2f}")
            print(f"系统硬限制: ${data.get('system_hard_limit_usd', 0):.2f}")
        else:
            print(f"无法获取订阅信息: {response.status_code}")
    except Exception as e:
        print(f"获取订阅信息时出错: {e}")
    
    # 检查使用情况
    try:
        usage_url = f"https://api.openai.com/v1/dashboard/billing/usage?start_date={start_date}&end_date={end_date}"
        response = requests.get(usage_url, headers=headers)
        
        if response.status_code == 200:
            data = response.json()
            total_usage = data.get('total_usage', 0) / 100  # 转换为美元
            
            print(f"\n=== 最近30天使用情况 ===")
            print(f"总使用金额: ${total_usage:.2f}")
            
            # 显示每日使用情况（最近7天）
            daily_costs = data.get('daily_costs', [])
            if daily_costs:
                print("\n最近7天详细使用:")
                for day in daily_costs[-7:]:
                    date = day.get('timestamp')
                    cost = day.get('line_items', [{}])[0].get('cost', 0) / 100
                    print(f"  {date}: ${cost:.2f}")
        else:
            print(f"无法获取使用情况: {response.status_code}")
    except Exception as e:
        print(f"获取使用情况时出错: {e}")

def test_api_with_simple_request():
    """用一个简单的请求测试API是否正常工作"""
    print("\n=== 测试API连接 ===")
    
    try:
        client = OpenAI()
        
        # 发送一个非常小的请求
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",  # 使用便宜的模型
            messages=[{"role": "user", "content": "Hi"}],
            max_tokens=1,
            temperature=0
        )
        
        print("✅ API连接正常！")
        print(f"响应: {response.choices[0].message.content}")
        
    except Exception as e:
        print(f"❌ API测试失败: {type(e).__name__}")
        print(f"错误信息: {str(e)}")
        
        # 判断错误类型
        error_str = str(e).lower()
        if "insufficient_quota" in error_str or "exceeded your current quota" in error_str:
            print("\n⚠️ 诊断: 您的API额度已用完，需要充值！")
        elif "invalid api key" in error_str or "incorrect api key" in error_str:
            print("\n⚠️ 诊断: API密钥无效！")
        elif "502" in error_str or "bad gateway" in error_str:
            print("\n⚠️ 诊断: OpenAI服务器临时故障，与余额无关")
        else:
            print("\n⚠️ 诊断: 其他错误，请检查网络连接和API设置")

def main():
    """主函数"""
    print("OpenAI API 状态检查工具")
    print("=" * 50)
    
    # 1. 检查环境变量
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        print("❌ 错误：未设置OPENAI_API_KEY环境变量")
        print("\n请运行：export OPENAI_API_KEY='your-api-key'")
        return
    
    print(f"✅ 找到API密钥 (前8位: {api_key[:8]}...)")
    
    # 2. 检查使用情况
    check_openai_usage()
    
    # 3. 测试API
    test_api_with_simple_request()
    
    print("\n" + "=" * 50)
    print("检查完成！")
    
    # 给出建议
    print("\n💡 建议：")
    print("1. 如果是余额不足，请前往 https://platform.openai.com/account/billing 充值")
    print("2. 如果是502错误，请等待几分钟后重试")
    print("3. 查看OpenAI服务状态: https://status.openai.com/")

if __name__ == "__main__":
    main()

In [ ]:
#!/usr/bin/env python3
"""
并行删除所有任务库中的 original_description 字段
使用 ThreadPoolExecutor 高效处理多个文件

使用方法:
    # 处理默认目录 (./mcp_generated_library/difficulty_versions)
    python remove_original_descriptions.py
    
    # 处理指定目录
    python remove_original_descriptions.py -d ./path/to/tasks
    
    # 同时处理父目录的任务文件
    python remove_original_descriptions.py -p
    
    # 自定义工作线程数
    python remove_original_descriptions.py -w 20
"""

import json
import logging
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
from typing import Dict, List, Tuple
from datetime import datetime

# 设置日志
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)


def process_single_file(file_path: Path) -> Tuple[str, int, int]:
    """
    处理单个任务文件，删除所有任务的 original_description 字段
    
    Args:
        file_path: 任务文件路径
        
    Returns:
        (文件名, 原始任务数, 修改的任务数)
    """
    logger.info(f"Processing file: {file_path}")
    
    # 读取文件
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # 统计信息
    total_tasks = 0
    modified_tasks = 0
    
    # 处理不同的数据格式
    if isinstance(data, dict):
        if 'tasks' in data:
            # 格式: {"tasks": [...]}
            tasks = data['tasks']
            total_tasks = len(tasks)
            
            for task in tasks:
                if 'original_description' in task:
                    del task['original_description']
                    modified_tasks += 1
                    
        else:
            # 可能是其他格式的字典
            logger.warning(f"Unexpected dict format in {file_path}")
            
    elif isinstance(data, list):
        # 格式: [task1, task2, ...]
        tasks = data
        total_tasks = len(tasks)
        
        for task in tasks:
            if isinstance(task, dict) and 'original_description' in task:
                del task['original_description']
                modified_tasks += 1
    
    # 保存修改后的文件
    if modified_tasks > 0:
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2, ensure_ascii=False)
        logger.info(f"✅ {file_path.name}: Modified {modified_tasks}/{total_tasks} tasks")
    else:
        logger.info(f"ℹ️ {file_path.name}: No original_description fields found")
    
    return (file_path.name, total_tasks, modified_tasks)


def find_all_task_files(directory: Path) -> List[Path]:
    """
    查找目录下所有的任务文件
    
    Args:
        directory: 要搜索的目录
        
    Returns:
        任务文件路径列表
    """
    task_files = []
    
    # 查找所有 .json 文件
    for file_path in directory.glob('*.json'):
        # 跳过一些明显不是任务文件的
        if any(skip in file_path.name.lower() for skip in ['config', 'registry', 'settings']):
            continue
        task_files.append(file_path)
    
    return task_files


def parallel_remove_original_descriptions(
    directory_path: str = "./mcp_generated_library/difficulty_versions",
    max_workers: int = 10
) -> Dict[str, any]:
    """
    并行处理目录下所有任务文件，删除 original_description 字段
    
    Args:
        directory_path: 任务文件目录路径
        max_workers: 最大并行工作线程数
        
    Returns:
        处理结果统计
    """
    start_time = time.time()
    directory = Path(directory_path)
    
    # 确保目录存在
    if not directory.exists():
        print(f"❌ Directory not found: {directory}")
        raise FileNotFoundError(f"Directory not found: {directory}")
    
    # 查找所有任务文件
    task_files = find_all_task_files(directory)
    
    if not task_files:
        print(f"⚠️ No task files found in {directory}")
        return {
            'total_files': 0,
            'total_tasks': 0,
            'modified_tasks': 0,
            'execution_time': 0
        }
    
    print(f"\n🔍 Found {len(task_files)} task files to process")
    print("=" * 60)
    
    # 结果统计
    results = {
        'total_files': len(task_files),
        'total_tasks': 0,
        'modified_tasks': 0,
        'file_results': []
    }
    
    # 使用线程池并行处理
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # 提交所有任务
        future_to_file = {
            executor.submit(process_single_file, file_path): file_path 
            for file_path in task_files
        }
        
        # 处理完成的任务
        completed = 0
        for future in as_completed(future_to_file):
            file_path = future_to_file[future]
            completed += 1
            
            # 获取结果
            file_name, total_tasks, modified_tasks = future.result()
            
            # 更新统计
            results['total_tasks'] += total_tasks
            results['modified_tasks'] += modified_tasks
            results['file_results'].append({
                'file': file_name,
                'total_tasks': total_tasks,
                'modified_tasks': modified_tasks
            })
            
            # 显示进度
            progress = (completed / len(task_files)) * 100
            print(f"Progress: {completed}/{len(task_files)} files ({progress:.1f}%)")
    
    # 计算执行时间
    execution_time = time.time() - start_time
    results['execution_time'] = execution_time
    
    # 显示最终结果
    print("\n" + "=" * 60)
    print("✨ Processing Complete!")
    print(f"📊 Total files processed: {results['total_files']}")
    print(f"📋 Total tasks processed: {results['total_tasks']}")
    print(f"✏️  Tasks modified: {results['modified_tasks']}")
    print(f"⏱️  Execution time: {execution_time:.2f} seconds")
    print("=" * 60)
    
    # 保存处理报告
    report_path = directory / f"original_description_removal_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    with open(report_path, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=2)
    print(f"\n📄 Report saved to: {report_path}")
    
    return results


if __name__ == "__main__":
    import argparse
    
    # 命令行参数解析
    parser = argparse.ArgumentParser(description='并行删除任务文件中的 original_description 字段')
    parser.add_argument(
        '--directory', '-d',
        default='./mcp_generated_library/difficulty_versions',
        help='任务文件目录路径 (默认: ./mcp_generated_library/difficulty_versions)'
    )
    parser.add_argument(
        '--workers', '-w',
        type=int,
        default=10,
        help='最大并行工作线程数 (默认: 10)'
    )
    parser.add_argument(
        '--include-parent', '-p',
        action='store_true',
        help='同时处理上级目录的任务文件'
    )
    
    args = parser.parse_args()
    
    # 执行主函数
    try:
        # 处理指定目录
        print(f"🚀 Processing directory: {args.directory}")
        results = parallel_remove_original_descriptions(
            directory_path=args.directory,
            max_workers=args.workers
        )
        
        # 如果需要，也处理上级目录
        if args.include_parent:
            parent_dir = Path(args.directory).parent
            print(f"\n🚀 Processing parent directory: {parent_dir}")
            parent_results = parallel_remove_original_descriptions(
                directory_path=str(parent_dir),
                max_workers=args.workers
            )
            
            # 合并结果
            results['total_files'] += parent_results['total_files']
            results['total_tasks'] += parent_results['total_tasks']
            results['modified_tasks'] += parent_results['modified_tasks']
            results['file_results'].extend(parent_results['file_results'])
        
        # 显示每个文件的详细结果
        print("\n📋 Detailed Results:")
        print("-" * 60)
        for file_result in results['file_results']:
            if file_result['modified_tasks'] > 0:
                print(f"✅ {file_result['file']}: {file_result['modified_tasks']}/{file_result['total_tasks']} tasks modified")
            else:
                print(f"ℹ️  {file_result['file']}: No modifications needed")
            
    except Exception as e:
        logger.error(f"❌ Error occurred: {str(e)}")
        raise  # 直接抛出异常，方便调试

In [ ]:
# 设置环境变量（在运行代码前）
import os

# 然后运行您的代码
from tool_and_task_generator import parallel_generate_tasks_from_existing_tools

results = parallel_generate_tasks_from_existing_tools(
    tool_registry_path="mcp_generated_library/tool_registry_consolidated.json",
    num_tasks=1000,
    task_distribution={
        'basic_task': 0.20,
        'simple_task': 0.20,
        'data_pipeline': 0.2,
        'api_integration': 0.2,
        'multi_stage_pipeline': 0.20
    },
    use_llm=True,  # 确保启用LLM
    max_workers=1000,
    show_progress=True
)

In [2]:
import json
import os
from pathlib import Path
from collections import defaultdict
import time

def merge_task_libraries(directory_path="mcp_generated_library/difficulty_versions", 
                        output_file="mcp_generated_library/task_library_all_difficulties.json"):
    """
    合并 difficulty_versions 目录下的所有 JSON 文件到一个文件
    
    Args:
        directory_path: 包含 JSON 文件的目录路径
        output_file: 输出合并后的 JSON 文件路径
    """
    
    print(f"🔍 扫描目录: {directory_path}")
    
    # 存储所有任务
    all_tasks = []
    
    # 统计信息
    stats = defaultdict(int)
    file_task_counts = {}
    
    # 获取目录下所有 .json 文件
    json_files = list(Path(directory_path).glob("*.json"))
    
    if not json_files:
        print(f"❌ 在 {directory_path} 中没有找到 JSON 文件")
        return
    
    print(f"📁 找到 {len(json_files)} 个 JSON 文件")
    print("=" * 50)
    
    # 处理每个 JSON 文件
    for json_file in sorted(json_files):
        file_name = json_file.name
        print(f"\n📄 处理文件: {file_name}")
        
        # 读取文件
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        # 提取任务
        tasks = []
        if isinstance(data, dict):
            if 'tasks' in data:
                tasks = data['tasks']
            else:
                # 如果是其他格式的字典，尝试提取值
                for value in data.values():
                    if isinstance(value, list):
                        tasks.extend(value)
        elif isinstance(data, list):
            tasks = data
        
        # 为每个任务添加来源文件信息
        for task in tasks:
            # 添加源文件标记
            task['source_file'] = file_name
            
            # 从文件名推断难度版本
            if 'very_easy' in file_name:
                task['difficulty_version'] = 'very_easy'
            elif 'easy_biased' in file_name:
                task['difficulty_version'] = 'easy_biased'
            elif 'easy' in file_name:
                task['difficulty_version'] = 'easy'
            elif 'medium_biased' in file_name:
                task['difficulty_version'] = 'medium_biased'
            elif 'medium' in file_name:
                task['difficulty_version'] = 'medium'
            elif 'hard_biased' in file_name:
                task['difficulty_version'] = 'hard_biased'
            elif 'hard' in file_name:
                task['difficulty_version'] = 'hard'
            elif 'very_hard' in file_name:
                task['difficulty_version'] = 'very_hard'
            else:
                task['difficulty_version'] = 'unknown'
            
            # 统计任务类型
            task_type = task.get('task_type', 'unknown')
            stats[f'task_type_{task_type}'] += 1
            
            # 统计复杂度
            complexity = task.get('complexity', 'unknown')
            stats[f'complexity_{complexity}'] += 1
            
            # 统计难度版本
            stats[f'difficulty_{task["difficulty_version"]}'] += 1
        
        # 添加到总任务列表
        all_tasks.extend(tasks)
        file_task_counts[file_name] = len(tasks)
        
        print(f"  ✓ 找到 {len(tasks)} 个任务")
    
    print("\n" + "=" * 50)
    print(f"📊 合并统计:")
    print(f"  总文件数: {len(json_files)}")
    print(f"  总任务数: {len(all_tasks)}")
    
    # 去重（基于任务ID）
    unique_tasks = {}
    duplicate_count = 0
    
    for task in all_tasks:
        task_id = task.get('id', None)
        if task_id:
            if task_id not in unique_tasks:
                unique_tasks[task_id] = task
            else:
                duplicate_count += 1
                # 如果是重复任务，记录所有出现的文件
                if 'all_source_files' not in unique_tasks[task_id]:
                    unique_tasks[task_id]['all_source_files'] = [unique_tasks[task_id].get('source_file')]
                unique_tasks[task_id]['all_source_files'].append(task.get('source_file'))
        else:
            # 没有ID的任务，使用其他字段生成唯一标识
            unique_key = f"{task.get('task_type', 'unknown')}_{task.get('complexity', 'unknown')}_{len(unique_tasks)}"
            unique_tasks[unique_key] = task
    
    print(f"  去重后任务数: {len(unique_tasks)}")
    print(f"  重复任务数: {duplicate_count}")
    
    # 创建最终的数据结构
    merged_data = {
        "metadata": {
            "generated_at": time.strftime('%Y-%m-%d %H:%M:%S'),
            "source_directory": directory_path,
            "total_files": len(json_files),
            "total_tasks": len(all_tasks),
            "unique_tasks": len(unique_tasks),
            "duplicates_removed": duplicate_count,
            "files_processed": list(file_task_counts.keys())
        },
        "statistics": {
            "by_file": file_task_counts,
            "by_task_type": {k.replace('task_type_', ''): v for k, v in stats.items() if k.startswith('task_type_')},
            "by_complexity": {k.replace('complexity_', ''): v for k, v in stats.items() if k.startswith('complexity_')},
            "by_difficulty_version": {k.replace('difficulty_', ''): v for k, v in stats.items() if k.startswith('difficulty_')}
        },
        "tasks": list(unique_tasks.values())
    }
    
    # 保存合并后的文件
    output_path = Path(output_file)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(merged_data, f, indent=2, ensure_ascii=False)
    
    print(f"\n✅ 合并完成！")
    print(f"📁 输出文件: {output_file}")
    
    # 显示详细统计
    print("\n📊 详细统计:")
    print("\n按文件统计:")
    for file_name, count in sorted(file_task_counts.items()):
        print(f"  {file_name}: {count} 个任务")
    
    print("\n按任务类型统计:")
    for task_type, count in sorted(merged_data['statistics']['by_task_type'].items()):
        print(f"  {task_type}: {count} 个任务")
    
    print("\n按复杂度统计:")
    for complexity, count in sorted(merged_data['statistics']['by_complexity'].items()):
        print(f"  {complexity}: {count} 个任务")
    
    print("\n按难度版本统计:")
    for difficulty, count in sorted(merged_data['statistics']['by_difficulty_version'].items()):
        print(f"  {difficulty}: {count} 个任务")
    
    return merged_data

# 执行合并
if __name__ == "__main__":
    # 使用默认参数运行
    result = merge_task_libraries()
    
    # 可选：显示前5个任务作为示例
    if result and 'tasks' in result:
        print("\n📋 示例任务（前5个）:")
        for i, task in enumerate(result['tasks'][:5]):
            print(f"\n任务 {i+1}:")
            print(f"  ID: {task.get('id', 'N/A')}")
            print(f"  类型: {task.get('task_type', 'N/A')}")
            print(f"  复杂度: {task.get('complexity', 'N/A')}")
            print(f"  难度版本: {task.get('difficulty_version', 'N/A')}")
            print(f"  来源文件: {task.get('source_file', 'N/A')}")

🔍 扫描目录: mcp_generated_library/difficulty_versions
📁 找到 8 个 JSON 文件

📄 处理文件: task_library_enhanced_v3_easy.json
  ✓ 找到 630 个任务

📄 处理文件: task_library_enhanced_v3_easy_biased.json
  ✓ 找到 630 个任务

📄 处理文件: task_library_enhanced_v3_hard.json
  ✓ 找到 630 个任务

📄 处理文件: task_library_enhanced_v3_hard_biased.json
  ✓ 找到 630 个任务

📄 处理文件: task_library_enhanced_v3_medium.json
  ✓ 找到 630 个任务

📄 处理文件: task_library_enhanced_v3_medium_biased.json
  ✓ 找到 630 个任务

📄 处理文件: task_library_enhanced_v3_very_easy.json
  ✓ 找到 630 个任务

📄 处理文件: task_library_enhanced_v3_very_hard.json
  ✓ 找到 630 个任务

📊 合并统计:
  总文件数: 8
  总任务数: 5040
  去重后任务数: 5040
  重复任务数: 0

✅ 合并完成！
📁 输出文件: mcp_generated_library/task_library_all_difficulties.json

📊 详细统计:

按文件统计:
  task_library_enhanced_v3_easy.json: 630 个任务
  task_library_enhanced_v3_easy_biased.json: 630 个任务
  task_library_enhanced_v3_hard.json: 630 个任务
  task_library_enhanced_v3_hard_biased.json: 630 个任务
  task_library_enhanced_v3_medium.json: 630 个任务
  task_library_enhanced_v3_medi

In [1]:
# 一行命令生成配置文件
import os, json
from pathlib import Path

# 确保设置了环境变量
# os.environ['OPENAI_API_KEY'] = 'sk-...'  # 取消注释并填入您的 key

# 创建配置
Path("./config").mkdir(exist_ok=True)
config = {
    "api_key": os.getenv('OPENAI_API_KEY', ''),
    "openai_api_key": os.getenv('OPENAI_API_KEY', ''),
    "model": "gpt-4o-mini"
}

# 保存配置
with open("./config/config.json", 'w') as f:
    json.dump(config, f, indent=2)

# 验证
if config['api_key']:
    print(f"✅ 配置成功! API Key: {config['api_key'][:8]}...")
else:
    print("❌ 请先设置 OPENAI_API_KEY 环境变量")

✅ 配置成功! API Key: sk-proj-...
